In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json
import pandas as pd
#import for threading
from concurrent.futures import ThreadPoolExecutor
import time
import numpy as np

#creation of the file
test = f"./fullTest.json"

#need to set up the driver outside of the scraping part bc we need several drivers
def driversetting():
    driver = webdriver.Chrome(executable_path="driver/chromedriver")
    return driver

def scraping(iteration, driver):
    list_of_properties = []

    for number in iteration:
        page_num = str(number) + "&orderBy=relevance"
        url = (
           "https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=" + page_num
        )

        driver.get(url)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        listings = soup.find_all("a", class_="card__title-link")
        for pages in listings:  

            driver.get(pages["href"]) # get the response from the url
            property_details_page = BeautifulSoup(driver.page_source, "html.parser")
            content = property_details_page.find("section", class_="classified_section")
            list_of_properties.append(content)
    return list_of_properties

def saving_datas(datas):
    with open(test, 'a') as file:
        json.dump(str(datas), file, indent=4)
#concurrency setting
def pool():
    # 1 driver by thread (we have four thread here)
    drivers = [driversetting() for _ in range(8)]
    #division of the number of pages by the number of threads 
    division = np.array_split(np.arange(1,8),8)
    #creation of a pool of threads
    with ThreadPoolExecutor(max_workers=8) as executor :
        results = executor.map(scraping,division,drivers)
        for result in results: 
            saving_datas(result)

pool()



C:\Users\feldm\AppData\Local\Temp\ipykernel_2276\4149297583.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="driver/chromedriver")
